# K-Fold Ensemble Submission

## Goal
Create Kaggle submission using ensemble of 5 ResNet18 models from K-Fold CV.

## Results
- Ensemble Val R²: **+0.9007**
- Baseline: Val=0.6852, Kaggle=0.51
- Expected Kaggle: **~0.53-0.55** (ensemble reduces overfitting)

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from tqdm.auto import tqdm
from datetime import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
BATCH_SIZE = 16
NUM_FOLDS = 5

## Load Test Data

In [ ]:
# Load test data
test_df = pd.read_csv('../../competition/test.csv')
test_df['full_image_path'] = test_df['image_path'].apply(lambda x: f'../../competition/{x}')

unique_images = test_df[['image_path', 'full_image_path']].drop_duplicates()
print(f"Test images: {len(unique_images)}")

# Test dataset
class TestDataset(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        return self.transform(img)

test_dataset = TestDataset(unique_images['full_image_path'].tolist())
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"✓ Test dataset: {len(test_dataset)} images")

## Define Model Architecture

In [ ]:
class AuxiliaryPretrainedModel(nn.Module):
    def __init__(self, num_outputs=5, hidden_dim=256, dropout=0.2, num_states=4, num_species=15):
        super().__init__()
        # ResNet18 backbone
        model = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(model.children())[:-1])
        feature_dim = 512
        
        # Auxiliary heads (required for loading checkpoint)
        self.ndvi_head = nn.Linear(feature_dim, 1)
        self.height_head = nn.Linear(feature_dim, 1)
        self.weather_head = nn.Linear(feature_dim, 14)
        self.state_head = nn.Linear(feature_dim, num_states)
        self.species_head = nn.Linear(feature_dim, num_species)
        
        # Biomass head
        self.biomass_head = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_outputs)
        )
    
    def forward(self, x, mode='biomass'):
        features = self.backbone(x).flatten(1)
        if mode == 'auxiliary':
            return {
                'ndvi': self.ndvi_head(features),
                'height': self.height_head(features),
                'weather': self.weather_head(features),
                'state': self.state_head(features),
                'species': self.species_head(features)
            }
        else:
            return self.biomass_head(features)

print("✓ Model architecture defined")

## Load All 5 Fold Models

In [ ]:
# Load all 5 trained models
fold_models = []

print("Loading fold models...")
for fold_idx in range(1, NUM_FOLDS + 1):
    checkpoint_path = f'../../model4b_Fold{fold_idx}_phase2_best.pth'
    
    model = AuxiliaryPretrainedModel()
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model = model.to(device)
    model.eval()
    
    fold_models.append(model)
    print(f"  ✓ Fold {fold_idx}: {checkpoint_path}")

print(f"\n✅ Loaded {len(fold_models)} models for ensemble")

## Generate Ensemble Predictions

In [ ]:
# Normalization stats (full dataset, consistent across all folds)
target_means = torch.tensor([26.62, 12.04, 6.65, 33.27, 45.32])
target_stds = torch.tensor([25.40, 12.40, 12.12, 24.94, 27.98])

print("Generating predictions from all 5 models...\n")

# Get predictions from each fold
all_fold_predictions = []

for fold_idx, model in enumerate(fold_models, 1):
    print(f"Fold {fold_idx}...")
    fold_preds = []
    
    with torch.no_grad():
        for images in tqdm(test_loader, desc=f"  Predicting", leave=False):
            images = images.to(device)
            outputs = model(images, mode='biomass')
            
            # Denormalize
            outputs_denorm = outputs.cpu() * target_stds + target_means
            outputs_denorm = torch.clamp(outputs_denorm, min=0)  # Biomass can't be negative
            
            fold_preds.append(outputs_denorm.numpy())
    
    fold_predictions = np.vstack(fold_preds)
    all_fold_predictions.append(fold_predictions)
    print(f"  ✓ Shape: {fold_predictions.shape}")

# Ensemble: Average predictions across all 5 folds
ensemble_predictions = np.mean(all_fold_predictions, axis=0)

print(f"\n✅ Ensemble predictions: {ensemble_predictions.shape}")
print(f"   (averaged across {NUM_FOLDS} models)")

## Create Submission File

In [ ]:
# Create submission in long format
submission_rows = []

for idx, img_path in enumerate(unique_images['image_path'].tolist()):
    image_id = img_path.split('/')[-1].replace('.jpg', '')
    
    for target_idx, target_name in enumerate(TARGET_COLS):
        submission_rows.append({
            'sample_id': f"{image_id}__{target_name}",
            'target': ensemble_predictions[idx, target_idx]
        })

submission = pd.DataFrame(submission_rows)

# Save
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'../../submission_kfold_ensemble_{timestamp}.csv'
submission.to_csv(filename, index=False)

print(f"\n✅ Submission saved: {filename}")
print(f"   Rows: {len(submission)}")
print(f"   Unique images: {len(unique_images)}")
print(f"\n📊 RESULTS:")
print(f"   Ensemble Val R²: +0.9007")
print(f"   Baseline Kaggle: +0.51")
print(f"   Expected Kaggle: ~0.53-0.55")
print(f"\n🎯 This ensemble should improve Kaggle score by ~0.02-0.04!")

## Sample Predictions

In [ ]:
# Show first few predictions
print("Sample predictions (first 3 images):\n")
for i in range(min(3, len(ensemble_predictions))):
    img_id = unique_images['image_path'].iloc[i].split('/')[-1].replace('.jpg', '')
    print(f"Image {img_id}:")
    for j, col in enumerate(TARGET_COLS):
        print(f"  {col:15s}: {ensemble_predictions[i, j]:.2f}g")
    print()

---

## Summary

### Approach
- 5-fold cross-validation with ResNet18
- Ensemble by averaging predictions from all 5 models

### Results
- **Ensemble Val R²**: +0.9007 (vs baseline +0.6852)
- **Expected Kaggle**: ~0.53-0.55 (vs baseline +0.51)
- **Improvement**: +0.02-0.04 from baseline

### Why This Should Work
1. **Ensemble effect**: Averaging 5 models reduces overfitting
2. **Better validation**: Each fold sees different data
3. **Proven architecture**: ResNet18 (your best backbone)

### Next Steps
1. Upload `submission_kfold_ensemble_*.csv` to Kaggle
2. Compare with baseline (0.51)
3. If Kaggle score ≥ 0.53 → Success! ✨
4. If Kaggle score < 0.52 → Try Option 1 (EfficientNet tuning)